In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('aqi.csv')
display(df.head())

,date,state,area,number_of_monitoring_stations,prominent_pollutants,aqi_value,air_quality_status,unit,note
0,30-04-2025,Maharashtra,Amravati,2,PM10,78,Satisfactory,number_of_monitoring_stations in Absolute Numb...,NaN
1,30-04-2025,Bihar,Purnia,1,CO,56,Satisfactory,number_of_monitoring_stations in Absolute Numb...,NaN
2,30-04-2025,Madhya Pradesh,Katni,1,O3,98,Satisfactory,number_of_monitoring_stations in Absolute Numb...,NaN
3,30-04-2025,Chhattisgarh,Tumidih,1,PM10,103,Moderate,number_of_monitoring_stations in Absolute Numb...,NaN
4,30-04-2025,Assam,Byrnihat,1,PM2.5,61,Satisfactory,number_of_monitoring_stations in Absolute Numb...,NaN


In [ ]:
print(df['state'].unique())

print()
print()
print(f'states {len(df['state'].unique())}')

print()
print()
print(df['area'].unique())

print()
print()
print(f'areas {len(df['area'].unique())}')

['Maharashtra' 'Bihar' 'Madhya Pradesh' 'Chhattisgarh' 'Assam' 'Manipur'
 'Kerala' 'West Bengal' 'Odisha' 'Karnataka' 'Gujarat' 'Uttarakhand'
 'Tamil Nadu' 'Andhra Pradesh' 'Rajasthan' 'Uttar Pradesh' 'Punjab'
 'Mizoram' 'Chandigarh' 'Telangana' 'Puducherry' 'Meghalaya'
 'Himachal Pradesh' 'Jharkhand' 'Haryana' 'Arunachal Pradesh' 'Nagaland'
 'Tripura' 'Delhi' 'Andaman and Nicobar Islands' 'Sikkim'
 'Jammu and Kashmir']


states 32


['Amravati' 'Purnia' 'Katni' 'Tumidih' 'Byrnihat' 'Imphal' 'Kollam'
 'Barrackpore' 'Nayagarh' 'Nalbari' 'Hubballi' 'Ahmedabad' 'Dehradun'
 'Vellore' 'Ulhasnagar' 'Chengalpattu' 'Tirupati' 'Dindigul' 'Kadapa'
 'Thane' 'Rishikesh' 'Raipur' 'Ranipet' 'Samastipur' 'Jabalpur'
 'Kishanganj' 'Saharsa' 'Yadgir' 'Dungarpur' 'Meerut' 'Sawai Madhopur'
 'Ghaziabad' 'Pithampur' 'Kanpur' 'Chennai' 'Nagaur' 'Chittoor' 'Kalyan'
 'Mira Bhayandar' 'Bhubaneswar' 'Churu' 'Jodhpur' 'Katihar' 'Pali'
 'Balasore' 'Sikar' 'Jalandhar' 'Tirumala' 'Angul' 'Patiala'
 'Pimpri Chinchwad

In [ ]:
print(f'Gujarat : {df.loc[df['state'] == 'Gujarat', ['area']]}')
print(f'Karnataka : {df.loc[df['state'] == 'Karnataka', ['area']]}')

Gujarat :                 area
11         Ahmedabad
51             Vatva
64      Gandhi Nagar
137            Surat
218       Ankleshwar
...              ...
235639         Vatva
235663     Nandesari
235685    Ankleshwar
235694  Gandhi Nagar
235744     Ahmedabad

[6215 rows x 1 columns]
Karnataka :                   area
10            Hubballi
27              Yadgir
52      Chamarajanagar
56      Chikkaballapur
61      Chikkamagaluru
...                ...
235759        Bagalkot
235763        Madikeri
235767          Mysuru
235773      Kalaburagi
235774          Koppal

[23494 rows x 1 columns]


In [ ]:
state_area_dict = (
    df.groupby('state')['area']
      .apply(lambda x: list(set(x)))
      .to_dict()
)
print(f'{state_area_dict}')

{'Andaman and Nicobar Islands': ['Sri Vijaya Puram'], 'Andhra Pradesh': ['Chittoor', 'Vijayawada', 'Anantapur', 'Tirumala', 'Rajamahendravaram', 'Amaravati', 'Visakhapatnam', 'Tirupati', 'Kadapa'], 'Arunachal Pradesh': ['Naharlagun'], 'Assam': ['Sivasagar', 'Nalbari', 'Silchar', 'Guwahati', 'Byrnihat', 'Nagaon'], 'Bihar': ['Gaya', 'Saharsa', 'Motihari', 'Manguraha', 'Darbhanga', 'Purnia', 'Begusarai', 'Samastipur', 'Aurangabad', 'Muzaffarpur', 'Arrah', 'Bettiah', 'Patna', 'Bhagalpur', 'Hajipur', 'Munger', 'Chhapra', 'Araria', 'Siwan', 'Katihar', 'Bihar Sharif', 'Kishanganj', 'Rajgir', 'Sasaram', 'Buxar'], 'Chandigarh': ['Chandigarh'], 'Chhattisgarh': ['Raipur', 'Bilaspur', 'Korba', 'Chhal', 'Tumidih', 'Milupara', 'Kunjemura', 'Bhilai'], 'Delhi': ['Delhi'], 'Gujarat': ['Vatva', 'Nandesari', 'Gandhi Nagar', 'Ahmedabad', 'Ankleshwar', 'Surat', 'Vapi'], 'Haryana': ['Ambala', 'Kaithal', 'Jind', 'Sirsa', 'Palwal', 'Dharuhera', 'Manesar', 'Sonipat', 'Fatehabad', 'Narnaul', 'Yamunanagar', 'Gur

In [ ]:
#  predicting for Delhi


import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# 1. Data Loading and Preprocessing
def get_area_data(df, state, area, lookback=30):
    # Filter by state and area
    data = df[(df['state'] == state) & (df['area'] == area)].copy()
    data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y')
    data = data.sort_values('date')

    # Resample to ensure daily continuity and handle missing values
    data = data.set_index('date')['aqi_value'].resample('D').mean().interpolate()

    if len(data) < lookback + 7:
        raise ValueError(f"Not enough data for {area}, {state} to create sequences.")

    return data

# 2. Define the LSTM Model
class AQILSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=7):
        super(AQILSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        # Take the output of the last time step and pass through linear layer
        out = self.fc(out[:, -1, :])
        return out

# 3. Training Function
def train_model(X, y, epochs=100, lr=0.01):
    X_tensor = torch.FloatTensor(X)
    y_tensor = torch.FloatTensor(y)

    model = AQILSTM()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        loss.backward()
        optimizer.step()

    return model

def predict_aqi(csv_path, state_name, area_name):
    df = pd.read_csv(csv_path)
    series = get_area_data(df, state_name, area_name)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_series = scaler.fit_transform(series.values.reshape(-1, 1))

    # Create Sequences (X: past 30 days, y: next 7 days)
    lookback = 30
    forecast = 7
    X, y = [], []
    for i in range(len(scaled_series) - lookback - forecast + 1):
        X.append(scaled_series[i : i + lookback])
        y.append(scaled_series[i + lookback : i + lookback + forecast].flatten())

    # Train
    model = train_model(np.array(X), np.array(y))

    # Predict for the next 7 days using the most recent 30 days
    last_window = torch.FloatTensor(scaled_series[-lookback:]).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        pred_scaled = model(last_window)
        predictions = scaler.inverse_transform(pred_scaled.numpy()).flatten()

    return predictions, series.values[-lookback:]

state = "Delhi"
area = "Delhi"
forecast_values, history = predict_aqi('new_aqi.csv', state, area)

print(f"Predicted AQI for next 7 days in {area}:")
print(forecast_values)

Predicted AQI for next 7 days in Delhi:
[228.86156 228.1207  228.80872 229.01472 230.065   228.58365 229.41962]


In [ ]:
###  pre-training on delhi
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

# 1. Define the Model Architecture (Must be the same during saving and loading)
class AQILSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=7):
        super(AQILSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# 2. Preparation and Training Function
def train_and_save_base_model(csv_path, state_name, area_name, save_path="aqi_pretrained_model.pth"):
    # Load Data
    df = pd.read_csv(csv_path)
    data = df[(df['state'] == state_name) & (df['area'] == area_name)].copy()
    data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y')
    series = data.sort_values('date').set_index('date')['aqi_value'].resample('D').mean().interpolate()

    # Normalize
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    # Create Sequences (30 days lookback -> 7 days forecast)
    lookback = 30
    forecast = 7
    X, y = [], []
    for i in range(len(scaled_data) - lookback - forecast + 1):
        X.append(scaled_data[i : i + lookback])
        y.append(scaled_data[i + lookback : i + lookback + forecast].flatten())

    X_train = torch.FloatTensor(np.array(X))
    y_train = torch.FloatTensor(np.array(y))

    # Initialize Model
    model = AQILSTM(input_size=1, hidden_size=64, num_layers=2, output_size=7)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training Loop
    print(f"Training pre-trained model on {area_name}...")
    model.train()
    for epoch in range(1000):  # Higher epochs for the base model
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 50 == 0:
            print(f'Epoch [{epoch+1}/200], Loss: {loss.item():.4f}')

    torch.save(model.state_dict(), save_path)
    print(f"Model saved successfully to {save_path}")

# pre-training for other areas
train_and_save_base_model('aqi.csv', 'Delhi', 'Delhi')

Training pre-trained model on Delhi...
Epoch [50/200], Loss: 0.0274
Epoch [100/200], Loss: 0.0198
Epoch [150/200], Loss: 0.0186
Epoch [200/200], Loss: 0.0179
Model saved successfully to aqi_pretrained_model.pth


In [10]:
# predict for other area
new_aqi= '../Dataset/new_aqi.csv'
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
class AQILSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=7):
        super(AQILSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def predict_new_area(model_path, csv_path, state_name, area_name):
    df = pd.read_csv(csv_path)
    data = df[(df['state'] == state_name) & (df['area'] == area_name)].copy()
    data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y')

    series = data.sort_values('date').set_index('date')['aqi_value'].resample('D').mean().interpolate()

    if len(series) < 30:
        raise ValueError("Need at least 30 days of historical data for this area.")

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    input_window = torch.FloatTensor(scaled_data[-30:]).unsqueeze(0)
    model = AQILSTM()
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval() 

    with torch.no_grad():
        prediction_scaled = model(input_window)


    prediction_actual = scaler.inverse_transform(prediction_scaled.numpy()).flatten()

    return prediction_actual

try:
    forecast = predict_new_area('../Model/aqi_pretrained_model.pth', new_aqi, 'Maharashtra', 'Mumbai')

    print("Predicted AQI for the next 7 days:")
    for i, val in enumerate(forecast, 1):
        print(f"Day {i}: {val:.2f}")
except Exception as e:
    print(f"Error: {e}")

Predicted AQI for the next 7 days:
Day 1: 121.89
Day 2: 123.14
Day 3: 123.39
Day 4: 127.36
Day 5: 128.38
Day 6: 128.14
Day 7: 130.83


In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, precision_score

# ---------------------------
# 1. MODEL ARCHITECTURE
# ---------------------------
class AQILSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=7):
        super(AQILSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# ---------------------------
# 2. UTILITY FUNCTIONS
# ---------------------------
def aqi_category(aqi):
    if aqi <= 50: return 0    # Good
    elif aqi <= 100: return 1 # Satisfactory
    elif aqi <= 200: return 2 # Moderate
    elif aqi <= 300: return 3 # Poor
    elif aqi <= 400: return 4 # Very Poor
    else: return 5            # Severe

def apply_exponential_calibration(preds, actuals, alpha=0.6):
    calibrated = preds.copy()
    for i in range(1, len(preds)):
        error = actuals[i - 1] - preds[i - 1]
        calibrated[i] = preds[i] + alpha * error
    return calibrated

def get_area_data(df, state, area, lookback=30):
    data = df[(df['state'] == state) & (df['area'] == area)].copy()
    data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y')
    data = data.sort_values('date')
    series = data.set_index('date')['aqi_value'].resample('D').mean().interpolate()
    return series

# ---------------------------
# 3. METRIC EVALUATION FUNCTION
# ---------------------------
def evaluate_model(model, X_test, y_test, scaler, tolerance=10, alpha=0.6):
    model.eval()
    with torch.no_grad():
        preds = model(X_test).cpu().numpy()

    # Inverse scaling to return to actual AQI values
    preds_real = scaler.inverse_transform(preds)
    y_real = scaler.inverse_transform(y_test.cpu().numpy())

    horizon = preds_real.shape[1]

    print("\n========= BASELINE METRICS (NO CALIBRATION) =========")
    baseline_rmses = []
    
    for day in range(horizon):
        rmse = np.sqrt(mean_squared_error(y_real[:, day], preds_real[:, day]))
        baseline_rmses.append(rmse)
        accuracy = np.mean(np.abs(y_real[:, day] - preds_real[:, day]) <= tolerance)
        print(f"Day-{day+1}: RMSE = {rmse:.1f}, ±{tolerance} AQI Accuracy = {accuracy*100:.1f}%")

    # Baseline Precision (Day 1)
    true_cat_day1 = [aqi_category(val) for val in y_real[:, 0]]
    pred_cat_day1 = [aqi_category(val) for val in preds_real[:, 0]]
    precision_base = precision_score(true_cat_day1, pred_cat_day1, average="weighted", zero_division=0)
    print(f"\nAQI Category Precision (Day-1): {precision_base:.2f}")

    print("\n========= CALIBRATED METRICS (EXPONENTIAL DECAY) =========")
    calibrated_preds = np.zeros_like(preds_real)
    for day in range(horizon):
        calibrated_preds[:, day] = apply_exponential_calibration(
            preds_real[:, day], y_real[:, day], alpha=alpha
        )

    # Reporting specific days as requested
    report_days = [0, 2, 6] # Day 1, 3, 7
    for day in report_days:
        if day < horizon:
            rmse_cal = np.sqrt(mean_squared_error(y_real[:, day], calibrated_preds[:, day]))
            improvement = (1 - rmse_cal / baseline_rmses[day]) * 100
            print(f"Day-{day+1}: RMSE = {rmse_cal:.1f}, Improvement = {improvement:.1f}%")

    print("\n========= CALIBRATED PRECISION METRICS =========")
    for day in report_days:
        if day < horizon:
            true_cat = [aqi_category(val) for val in y_real[:, day]]
            cal_cat = [aqi_category(val) for val in calibrated_preds[:, day]]
            precision_cal = precision_score(true_cat, cal_cat, average="weighted", zero_division=0)
            print(f"Day-{day+1} Calibrated Precision: {precision_cal:.3f}")

    print("\n===============================================")

# ---------------------------
# 4. EXECUTION FLOW
# ---------------------------
# Configuration - Adjust paths if necessary
CSV_PATH = '../Dataset/new_aqi.csv' 
MODEL_PATH = '../Model/aqi_pretrained_model.pth' # Generated from cell In[9] of your notebook

try:
    # Prepare Data (Example using Delhi as per notebook)
    df = pd.read_csv(CSV_PATH)
    series = get_area_data(df, "Delhi", "Delhi")
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_series = scaler.fit_transform(series.values.reshape(-1, 1))

    lookback, forecast = 30, 7
    X, y = [], []
    for i in range(len(scaled_series) - lookback - forecast + 1):
        X.append(scaled_series[i : i + lookback])
        y.append(scaled_series[i + lookback : i + lookback + forecast].flatten())

    X_test_tensor = torch.FloatTensor(np.array(X))
    y_test_tensor = torch.FloatTensor(np.array(y))

    # Initialize and Load Model
    model = AQILSTM()
    model.load_state_dict(torch.load(MODEL_PATH, weights_only=True))

    # Run Evaluation
    evaluate_model(model, X_test_tensor, y_test_tensor, scaler)

except FileNotFoundError:
    print(f"Error: Ensure '{CSV_PATH}' and '{MODEL_PATH}' exist in the directory.")
except Exception as e:
    print(f"An error occurred: {e}")


========= BASELINE METRICS (NO CALIBRATION) =========
Day-1: RMSE = 35.6, ±10 AQI Accuracy = 23.7%
Day-2: RMSE = 36.3, ±10 AQI Accuracy = 24.0%
Day-3: RMSE = 35.8, ±10 AQI Accuracy = 24.9%
Day-4: RMSE = 36.4, ±10 AQI Accuracy = 26.0%
Day-5: RMSE = 37.0, ±10 AQI Accuracy = 24.8%
Day-6: RMSE = 37.0, ±10 AQI Accuracy = 23.0%
Day-7: RMSE = 39.0, ±10 AQI Accuracy = 20.5%

AQI Category Precision (Day-1): 0.72

========= CALIBRATED METRICS (EXPONENTIAL DECAY) =========
Day-1: RMSE = 40.9, Improvement = -14.7%
Day-3: RMSE = 35.5, Improvement = 0.9%
Day-7: RMSE = 36.1, Improvement = 7.3%

========= CALIBRATED PRECISION METRICS =========
Day-1 Calibrated Precision: 0.686
Day-3 Calibrated Precision: 0.711
Day-7 Calibrated Precision: 0.712

